In [1]:
import tensorflow as tf

import CNN # External class generated for CNN

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Disable eager execution
tf.compat.v1.disable_eager_execution()


epochs=100
batch_size=256


eps_FSGM=0.10

# Select the attacker
#White-box: "FGSM", "BIM", "PGD", 
#Black-box: "SA","ZA"
attack_type = "SA"
dataset="CIRFAR10"   # MNIST, CIRFAR10

#path='results/MNIST_BA.txt' #this for MNIST images
path='results/CIRFAR10/CIRFAR10_SA.txt' 

In [2]:
# Load CIFAR-10 dataset
from tensorflow.keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

min_pixel_value = 0.0
max_pixel_value = 1.0


t_consumed=0
classifier, t_consumed = CNN.train(train_images, train_labels, epochs, batch_size,
                                   min_pixel_value, max_pixel_value, test_images, test_labels)

'''# Save the model
classifier.save("/home/app/Desktop/paper 6/code_CIFAR10/classifier.h5")'''

Instructions for updating:
Colocations handled automatically by placer.


/home/app/.local/lib/python3.8/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Train on 50000 samples, validate on 10000 samples
Epoch 1/75
50000/50000 [==============================] - ETA: 0s - loss: 1.7113 - accuracy: 0.3702

/home/app/.local/lib/python3.8/site-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 [==============================] - 110s 2ms/sample - loss: 1.7113 - accuracy: 0.3702 - val_loss: 2.5522 - val_accuracy: 0.2284
Epoch 2/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 1.2303 - accuracy: 0.5645 - val_loss: 1.1015 - val_accuracy: 0.6148
Epoch 3/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 1.0090 - accuracy: 0.6525 - val_loss: 1.0406 - val_accuracy: 0.6439
Epoch 4/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.8685 - accuracy: 0.7044 - val_loss: 0.7623 - val_accuracy: 0.7405
Epoch 5/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.7732 - accuracy: 0.7396 - val_loss: 0.8069 - val_accuracy: 0.7263
Epoch 6/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.6924 - accuracy: 0.7693 - val_loss: 0.6370 - val_accuracy: 0.7841
Epoch 7/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.6339 - accuracy: 0.7905 - v

50000/50000 [==============================] - 109s 2ms/sample - loss: 0.1061 - accuracy: 0.9653 - val_loss: 0.6996 - val_accuracy: 0.8492
Epoch 56/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.0960 - accuracy: 0.9684 - val_loss: 0.8500 - val_accuracy: 0.8259
Epoch 57/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.0953 - accuracy: 0.9687 - val_loss: 0.7817 - val_accuracy: 0.8404
Epoch 58/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.0924 - accuracy: 0.9706 - val_loss: 0.7606 - val_accuracy: 0.8484
Epoch 59/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.0933 - accuracy: 0.9704 - val_loss: 0.7878 - val_accuracy: 0.8503
Epoch 60/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.0898 - accuracy: 0.9717 - val_loss: 0.7379 - val_accuracy: 0.8537
Epoch 61/75
50000/50000 [==============================] - 109s 2ms/sample - loss: 0.0902 - accuracy: 0.97

'# Save the model\nclassifier.save("/home/app/Desktop/paper 6/code_CIFAR10/classifier.h5")'

In [3]:
for i in range(5, 55, 5):
    eps_FSGM= (i/100)
    print("..............EPS..........: ", eps_FSGM)
    #Evaluate the model
    acc, prc, rec, f1 = CNN.test(classifier, test_images, test_labels)

    #write the results
    import Writer
    results ="===========\n" +  dataset + "-Regular:\n acc, prc, rec, f1, epochs, batch size, time: acc, prc, rec, f1\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(epochs)+ "," + str(batch_size) + "," + str(int(t_consumed)) +"\n" 
    Writer.write_results(results, path)
    #==================================
    
    # Extract the features for the regular images
    from tensorflow.keras import models

    model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_features').output)
    Features_regular = model_output.predict(test_images)
    print(Features_regular.shape)   
    #==================================

    # Generate adversarial examples using the FastGradientMethod attack
    from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent #white-box
    from art.attacks.evasion import ZooAttack, SquareAttack, BoundaryAttack #black-box


    # Initialize the attack
    attack = None

    # Select the attack
    if attack_type == "FGSM":
        attack = FastGradientMethod(estimator=classifier, eps=eps_FSGM)
    if attack_type == "BIM":
        attack = BasicIterativeMethod(estimator=classifier, eps=eps_FSGM)
    elif attack_type == "PGD":
        attack = ProjectedGradientDescent(estimator=classifier, eps=eps_FSGM)
    elif attack_type == "SA":
        attack = SquareAttack(estimator=classifier,eps=eps_FSGM)
    elif attack_type == "ZA":
        attack = ZooAttack(classifier=classifier, max_iter=2, learning_rate=eps_FSGM)

    adv_images = attack.generate(test_images)
    #=============================================
    
    # Evaluate the model on the adversarial test set images
    acc, prc, rec, f1 = CNN.test(classifier, adv_images, test_labels)

    #write the results
    results =dataset + "-Adversarial:\n acc, prc, rec, f1, epochs, attack_type, eps_FSGM\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + attack_type + "," + str(eps_FSGM) +"\n" 
    Writer.write_results(results, path)
    #=============================================

    # Extract the features for the adversarial images
    model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_features').output)
    Features_adversarial = model_output.predict(adv_images)
    print(Features_adversarial.shape)
    #=============================================

    # merge regular and feature test images
    from sklearn.model_selection import train_test_split

    #merge data features of regular and adversarial images
    reg_adv_images = np.concatenate((Features_regular, Features_adversarial), axis=0)

    # generate labels for regular and adversarial, and merge them

    label_reg = np.zeros((10000, 1))
    label_adv = np.ones((10000, 1))
    reg_adv_labels=np.concatenate((label_reg, label_adv), axis=0)

    print("data details: ", reg_adv_images.shape, " , labels: ", reg_adv_labels.shape)

    X_train, X_test, y_train, y_test = train_test_split(reg_adv_images, reg_adv_labels, test_size=0.20, random_state=42)

    #fractin to use a part of the merged regular and adversarial images
    fraction=1
    #reduce the length because the memory is out
    X_train=X_train[:int(X_train.shape[0]/fraction)]
    X_test=X_test[:(int(X_test.shape[0]/fraction))]
    y_train=y_train[:(int(y_train.shape[0]/fraction))]
    y_test=y_test[:(int(y_test.shape[0]/fraction))]
    X_train, X_test, y_train, y_test
    print("data after split : X_train, X_test, y_train, y_test\n", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    #=============================================

    # integrate the adversarial detector model to CNN to exclude adversarial images or outlier images.

    size_advs=10000
    test_reg_adv_images=np.concatenate((test_images,adv_images[:size_advs]), axis=0)
    test_reg_adv_labels=np.concatenate((test_labels,test_labels[:size_advs]), axis=0)
    print(test_reg_adv_images.shape, test_reg_adv_labels.shape)

    # Shuffle the data
    indices = np.arange(len(test_reg_adv_images))
    np.random.shuffle(indices)
    test_reg_adv_images= test_reg_adv_images[indices]
    test_reg_adv_labels = test_reg_adv_labels[indices]

    # Evaluate the model after integrate regular and adversarial images
    acc, prc, rec, f1 = CNN.test(classifier, test_reg_adv_images, test_reg_adv_labels)
    results =dataset + "-Regular+Adversarial:\n acc, prc, rec, f1\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) +"\n" 
    Writer.write_results(results, path)

    #=============================================

    # Extract features 
    model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_features').output)
    Features_adv_reg = model_output.predict(test_reg_adv_images)
    print(Features_adversarial.shape)
    #=============================================

    #to ensure that the size of regular and adversarial image size are the same to do fair evaluation
    def make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels):
        diff=int(np.absolute(clean_images.shape[0] - attacked_images.shape[0])/2)
        size=int((clean_images.shape[0]+attacked_images.shape[0])/2)
        temp_clean_images=np.ones((size,32,32,3))
        temp_attacked_images=np.ones((size,32,32,3))

        temp_clean_labels=np.ones((size,1))
        temp_attacked_labels=np.ones((size,1))

        if(clean_images.shape[0]>attacked_images.shape[0]):
            temp_clean_images=clean_images[:size]
            temp_clean_labels=clean_labels[:size]

            temp_attacked_images=np.concatenate((attacked_images,clean_images[-diff:]), axis=0)
            temp_attacked_labels=np.concatenate((temp_attacked_labels,clean_labels[-diff:]), axis=0)

        elif(clean_images.shape[0]<attacked_images.shape[0]):
            temp_attacked_images=attacked_images[:size]
            temp_attacked_labels=attacked_labels[:size]

            temp_clean_images=np.concatenate((clean_images,attacked_images[-diff:]), axis=0)
            temp_clean_labels=np.concatenate((clean_labels,attacked_labels[-diff:]), axis=0)

        clean_images=temp_clean_images
        clean_labels=temp_clean_labels

        attacked_images=temp_attacked_images
        attacked_labels=temp_attacked_labels

        #print("clean_images.shape, attacked_images.shape", clean_images.shape, attacked_images.shape)
        #print("clean_labels.shape, attacked_labels.shape", clean_labels.shape, attacked_labels.shape)

        return clean_images,clean_labels, attacked_images, attacked_labels

    #Exclude adversarial images from testing and use only the regular images

    def retrurn_clean_images(adv_images_pred_binary):

        # Count the number of times 1 appears in the array
        number_of_1s = np.count_nonzero(adv_images_pred_binary == 1)
        # Count the number of times 0 appears in the array
        number_of_0s = np.count_nonzero(adv_images_pred_binary == 0)
        print("The number of attacked immages 1 is:", number_of_1s)
        print("The number of clean images 0 is:", number_of_0s)

        #Exclude adversarial images from testing and use only the regular images
        # Create a boolean mask use only the clean images
        mask_clean = np.where(adv_images_pred_binary == 0)[0]
        mask_adv = np.where(adv_images_pred_binary == 1)[0]

        # Exclude adversarial images
        #clean images
        clean_images = test_reg_adv_images[mask_clean]
        clean_labels = test_reg_adv_labels[mask_clean]
        #Adversarial images,attacked images
        attacked_images = test_reg_adv_images[mask_adv]
        attacked_labels = test_reg_adv_labels[mask_adv]

        clean_images,clean_labels, attacked_images, attacked_labels = make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels)

        print("clean_images:", clean_images.shape)
        print("clean_labels:", clean_labels.shape)

        return clean_images, clean_labels


    def eval_model_after_exclude_adv_image(model_name, new_test_clean_images, new_test_clean_labels, t_consumed):# Evaluate the model after excluding the adversarial images
        acc, prc, rec, f1 = CNN.test(classifier, new_test_clean_images, new_test_clean_labels)
        results =model_name + ":\n acc, prc, rec, f1, time_detection\n"
        results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(int(t_consumed)) + "\n" 
        Writer.write_results(results, path)
    #=============================================

    #Evaluate the mosdel after inegrate NN as outlier detection to exclude Adversarial images
    import ML_Classifiers as MLC
    t_str=time.time()

    adv_images_pred_binary= MLC.NN(X_train, X_test, y_train, y_test,eps_FSGM, attack_type, Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+NN_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate RF as outlier detection to exclude Adversarial images
    import ML_Classifiers as MLC
    t_str=time.time()

    adv_images_pred_binary= MLC.Random_Forest(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+RF_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)

    #=============================================

    ##Evaluate the mosdel after inegrate DT as outlier detection to exclude Adversarial images
    t_str=time.time()
    adv_images_pred_binary= MLC.DT(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+DT_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate KNN as outlier detection to exclude Adversarial images
    t_str=time.time()
    adv_images_pred_binary= MLC.KNN(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+KNN_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate GBM as outlier detection to exclude Adversarial images
    t_str=time.time()

    adv_images_pred_binary= MLC.GBM(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+GBM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate XGB as outlier detection to exclude Adversarial images
    t_str=time.time()

    adv_images_pred_binary= MLC.XGB(X_train, X_test, y_train, y_test,Features_adv_reg)

    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+XGB_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)
    #=============================================

..............EPS..........:  0.05


/home/app/.local/lib/python3.8/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


acc, prc, rec, f1: 85.31 85.33 85.31 85.28
(10000, 32)



acc, prc, rec, f1: 32.53 37.48 32.53 32.41
(10000, 32)
data details:  (20000, 32)  , labels:  (20000, 1)
data after split : X_train, X_test, y_train, y_test
 (16000, 32) (4000, 32) (16000, 1) (4000, 1)
(20000, 32, 32, 3) (20000, 1)
acc, prc, rec, f1: 58.92 61.9 58.92 59.38
(10000, 32)
Train on 16000 samples
Epoch 1/50
16000/16000 [==============================] - 0s 20us/sample - loss: 0.6252 - accuracy: 0.6884
Epoch 2/50
16000/16000 [==============================] - 0s 16us/sample - loss: 0.5557 - accuracy: 0.7344
Epoch 3/50
16000/16000 [==============================] - 0s 16us/sample - loss: 0.5317 - accuracy: 0.7532
Epoch 4/50
16000/16000 [==============================] - 0s 16us/sample - loss: 0.5168 - accuracy: 0.7591
Epoch 5/50
16000/16000 [==============================] - 0s 16us/sample - loss: 0.5089 - accuracy: 0.7642
Epoch 6/50
16000/16000 [==============================] - 0s 16us/sample - loss: 0.4978 - accuracy: 0.7701
Epoch 7/50
16000/16000 [========================

/home/app/Desktop/paper 6/code_CIFAR10/ML_Classifiers.py:252: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


...

GBM detection: acc, prc, rec, f1: 98.22 98.24 98.22 98.22
The number of attacked immages 1 is: 9828
The number of clean images 0 is: 10172
clean_images: (10000, 32, 32, 3)
clean_labels: (10000, 1)


/home/app/Desktop/paper 6/code_CIFAR10/ML_Classifiers.py:200: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


acc, prc, rec, f1: 83.58 83.41 83.58 83.44
XGB detection: acc, prc, rec, f1: 93.82 94.41 93.82 93.8
The number of attacked immages 1 is: 8689
The number of clean images 0 is: 11311


/home/app/Desktop/paper 6/code_CIFAR10/ML_Classifiers.py:161: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


clean_images: (10000, 32, 32, 3)
clean_labels: (10000, 1)
acc, prc, rec, f1: 75.54 75.48 75.54 75.14


In [4]:
##Evaluate the mosdel after inegrate FAM as outlier detection to exclude Adversarial images

for i in range(5, 55, 5):
    eps_FSGM= (i/100)
    print("..............EPS..........: ", eps_FSGM)

    import FAM 

    t_str=time.time()
    ArtMap=FAM.train(X_train[:5000], y_train[:5000])

    adv_images_pred_binary = FAM.test(X_test[:1000], y_test[:1000], ArtMap, Features_adv_reg)

    adv_images_pred_binary=np.array(adv_images_pred_binary)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+FAM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)




..............EPS..........:  0.05
859  values were replaced from  1000


TypeError: 'int' object is not subscriptable